# Задача 1

Пусть $X_1, X_2, \ldots, X_n$ — выборка из экспоненциального распределения с параметром $\lambda$. Найти оценку максимального правдоподобия параметра $\lambda$, сравнить ее с байесовской оценкой (MAP и математическое ожидание апостреорного распределения), подобрав сопряженное распределение. Сравнить полученные байесовские оценки с оценкой MLE. Найти предсказательное распределение

### Решение задачи 1

Разбираем экспоненциальную модель: выводим правдоподобие, MLE, байесовские оценки с гамма-приором и предсказательное распределение.

**Шаги расчёта**

1. Правдоподобие экспоненциальной выборки
$$
 f(x\mid \lambda)=\lambda e^{-\lambda x},\qquad L(\lambda \mid \mathbf{x})=\lambda^n \exp\!\left(-\lambda\sum_{i=1}^n x_i\right).
$$
Логарифм:
$$
 \ell(\lambda)= n\log \lambda - \lambda\sum_{i=1}^n x_i.
$$
Максимум достигается при
$$
 \hat{\lambda}_{\text{MLE}}=\frac{n}{\sum_{i=1}^n x_i}=\frac{1}{\bar x}.
$$

2. Априорное распределение
$$
 \lambda \sim \Gamma(\kappa, \tau), \qquad \pi(\lambda)=\frac{\tau^{\kappa}}{\Gamma(\kappa)}\lambda^{\kappa-1}e^{-\tau\lambda},\; \kappa,\tau>0.
$$

3. Апостериорное распределение
$$
 \lambda \mid \mathbf{x} \sim \Gamma\big(\kappa+n,\; \tau+\textstyle\sum_{i=1}^n x_i\big).
$$

4. Байесовские точечные оценки
$$
 \hat{\lambda}_{\text{MAP}}=\frac{\kappa+n-1}{\tau+\sum x_i},\qquad
 \hat{\lambda}_{\text{mean}}=\mathbb{E}[\lambda\mid\mathbf{x}]=\frac{\kappa+n}{\tau+\sum x_i}.
$$

5. Предсказательное распределение для нового значения $y$
$$
 p(y\mid\mathbf{x}) = \int_0^{\infty} \lambda e^{-\lambda y} \; \Gamma(\lambda\mid\kappa+n,\tau+\sum x_i)\, d\lambda =
 \frac{(\kappa+n)\, (\tau+\sum x_i)^{\kappa+n}}{(y+\tau+\sum x_i)^{\kappa+n+1}}.
$$
Это распределение Ломакса (Парето второго рода) с тяжёлым хвостом.

6. Поведение оценок
При малых выборках байесовские оценки тянутся к априорному среднему $\kappa/\tau$, а при большом $n$ все подходы сходятся к $1/\bar x$.

In [1]:
import numpy as np

rng = np.random.default_rng(7)
true_rate = 0.6
sample = rng.exponential(scale=1.0 / true_rate, size=25)

# Априорные гиперпараметры (shape, rate)
kappa0, tau0 = 1.5, 0.8

n = sample.size
sum_x = sample.sum()

lambda_mle = n / sum_x
kappa_post = kappa0 + n
tau_post = tau0 + sum_x
lambda_map = (kappa_post - 1) / tau_post
lambda_mean = kappa_post / tau_post

# Сэмплы из апостериора (numpy использует scale = 1/rate)
posterior_draws = rng.gamma(shape=kappa_post, scale=1.0 / tau_post, size=5000)
ci_low, ci_high = np.quantile(posterior_draws, [0.05, 0.95])

# Генерируем будущие наблюдения, учитывая неопределённость в lambda
lambda_draws = rng.gamma(shape=kappa_post, scale=1.0 / tau_post, size=3000)
y_pred = rng.exponential(scale=1.0 / lambda_draws)

print(f"Истинная интенсивность: {true_rate:.3f}")
print(f"MLE: {lambda_mle:.3f}")
print(f"MAP: {lambda_map:.3f}")
print(f"Posterior mean: {lambda_mean:.3f}")
print(f"90% доверительный интервал для lambda: [{ci_low:.3f}, {ci_high:.3f}]")
print(f"Среднее предсказание для нового наблюдения: {y_pred.mean():.3f}")

Истинная интенсивность: 0.600
MLE: 0.604
MAP: 0.604
Posterior mean: 0.628
90% доверительный интервал для lambda: [0.438, 0.844]
Среднее предсказание для нового наблюдения: 1.689


**Итоги задачи 1.** MLE использует только данные и эквивалентен обратному выборочному среднему. Гамма-приор даёт сглаженные MAP/mean оценки и тяжёлохвостое предсказательное распределение: оно реже недооценивает редкие большие времена ожидания, что полезно при малых выборках.

# Задача 2

**Мультиномиальное распределение**

Пусть проводится серия из $n$ испытаний и в результате каждого испытания происходит ровно одно событие из набора $A_1, A_2, \dots, A_m$, причем вероятности этих событий равны соответственно $\mathsf{p}_1, \mathsf{p}_2, \dots, \mathsf{p}_m$, причем
$$\sum_{i=1}^{m}\mathsf{p}_i = 1.$$

Тогда совместное распределение величин $X_1, X_2, \dots, X_m$, где $X_k$ — число наступлений события $A_k$ в серии из $n$ испытаний, задается вероятностями

$$
\mathsf{P}\left(X_1 = n_1, \dots, X_m = n_m, \right) = \frac{n!}{n_1!\dots n_m!}\mathsf{p}_1^{n_1}\dots \mathsf{p}_m^{n_m},
$$

где $n_1, n_2, \dots, n_m$ — произвольный набор целых неотрицательных чисел, таких что

$$\sum_{i=1}^m n_i = n.$$

Произведите байесовский вывод для мультиномиального распределения: найдите апостериорное распределение, используя в качестве сопоряженного распределения к правдоподобию [распределение Дирихле](https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D1%81%D0%BF%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5_%D0%94%D0%B8%D1%80%D0%B8%D1%85%D0%BB%D0%B5), найдите предсказательное распределение. Объясните результат.

### Байесовский вывод для мультиномиальной модели

Работаем с набором $m$ категорий. Будем обновлять вероятности исходов при помощи сопряжённого приора Дирихле и строить предсказания для новых экспериментов.

**Формулы**

Априор для вектора вероятностей $\mathbf{p}$:
$$
 \mathbf{p} \sim \mathrm{Dir}(\alpha_1,\dots,\alpha_m), \quad \alpha_k>0.
$$

Правдоподобие наблюдённых счётчиков $\mathbf{n}=(n_1,\dots,n_m)$, где $\sum n_k = n$:
$$
 L(\mathbf{p}\mid\mathbf{n}) \propto \prod_{k=1}^m p_k^{n_k}.
$$

После наблюдений апостериор снова Дирихле:
$$
 \mathbf{p}\mid\mathbf{n} \sim \mathrm{Dir}(\alpha_1+n_1,\dots,\alpha_m+n_m).
$$

Предсказание одного следующего испытания:
$$
 \mathbb{P}(A_k\mid\mathbf{n}) = \frac{\alpha_k+n_k}{\sum_{j=1}^m (\alpha_j+n_j)}.
$$

Для пакета из $t$ будущих испытаний с итоговыми счётчиками $\mathbf{u}$ (\(\sum u_k = t\)) получается распределение Дирихле–мультиномиальное:
$$
 p(\mathbf{u}\mid\mathbf{n}) = \frac{t!}{\prod_{k=1}^m u_k!} \, \frac{B(\boldsymbol{\alpha}+\mathbf{n}+\mathbf{u})}{B(\boldsymbol{\alpha}+\mathbf{n})},
$$
где $B(\mathbf{a}) = \dfrac{\prod_k \Gamma(a_k)}{\Gamma(\sum_k a_k)}$.

**Интерпретация**
Априорные параметры $\alpha_k$ работают как псевдосчётчики. Ненулевой $\alpha_k$ сглаживает нули в данных, а при росте числа наблюдений априорное влияние исчезает.

In [2]:
import numpy as np
from scipy.special import gammaln

rng = np.random.default_rng(3)

# Априор: слабая уверенность в равномерных вероятностях (по 0.5 псевдосчёта)
alpha0 = np.array([0.5, 0.5, 0.5, 0.5])
# Наблюдённые количества
obs = np.array([12, 5, 3, 0])

alpha_post = alpha0 + obs
post_mean = alpha_post / alpha_post.sum()

# Сэмплинг из апостериора для доверительных интервалов
samples_p = rng.dirichlet(alpha_post, size=4000)
ci = np.quantile(samples_p, [0.05, 0.95], axis=0)

# Предсказание одного опыта
pred_one = post_mean

# Функция вероятности для будущих t испытаний (Дирихле–мультиномиал)
def dirichlet_multinomial_prob(future_counts, alpha):
    future_counts = np.asarray(future_counts)
    t = future_counts.sum()
    term1 = gammaln(t + 1) - gammaln(future_counts + 1).sum()
    term2 = gammaln(alpha.sum()) - gammaln(alpha.sum() + t)
    term3 = (gammaln(alpha + future_counts) - gammaln(alpha)).sum()
    return np.exp(term1 + term2 + term3)

future = np.array([1, 1, 1, 0])
p_future = dirichlet_multinomial_prob(future, alpha_post)

print('Апостериорные параметры:', alpha_post)
print('Предсказание одного испытания (сглаженные частоты):', np.round(pred_one, 3))
for i, (lo, hi) in enumerate(zip(ci[0], ci[1]), 1):
    print(f'  p{i} 90% CI: [{lo:.3f}, {hi:.3f}]')
print(f'Вероятность будущего набора {future.tolist()} (t={future.sum()}): {p_future:.4f}')

Апостериорные параметры: [12.5  5.5  3.5  0.5]
Предсказание одного испытания (сглаженные частоты): [0.568 0.25  0.159 0.023]
  p1 90% CI: [0.399, 0.735]
  p2 90% CI: [0.115, 0.410]
  p3 90% CI: [0.055, 0.297]
  p4 90% CI: [0.000, 0.088]
Вероятность будущего набора [1, 1, 1, 0] (t=3): 0.1189


**Итоги задачи 2.** Обновление Дирихле сводится к сложению параметров с наблюдаемыми счётчиками. Предсказательные вероятности — это сглаженные частоты, которые не дают нулей даже при отсутствующих категориях. Для набора будущих испытаний вероятность задаётся Дирихле–мультиномиалом, объединяющим факториалы и отношения гамма-функций.